In [1]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json 
import requests
import datetime
from datetime import date
import warnings
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 800)

In [2]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='a539d7df2c7b43e1ac4d12f386d901e8')

In [3]:
x = newsapi.get_top_headlines(country='us', page_size=50, category = 'health')

In [4]:
nyt_api_key = '13bd501bc77542a58e2e6678619b0d60'

In [5]:
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

# NYT & NewsAPI

In [18]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
import datetime
from datetime import date
from apikeys import *
from info import *

#dates to use for API call
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

#NEW YORK TIMES
#clean the response from NYT API
def NYT_title_clean(df):
    titles = []
    for index, row in df.iterrows():
        title = row.headline['main']
        titles.append(title)
    df['title'] = titles
    return df

def NYT_dropped_rows(df):
    df.pub_date = pd.to_datetime(df.pub_date).dt.date
    df.word_count = round(df.word_count / 150)
    df.document_type = 'text'
    df['formality'] = 'Intermediate'
    return df

def NYT_dataframe_clean(df):
    dataframe = NYT_title_clean(df)
    dataframe = NYT_dropped_rows(dataframe)
    return dataframe

def NYT_api_call_section_based(section, source, page, start, end, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:({section_name})&page={page}&source:({source})&begin_date={start}&end_date={end}&api-key={api}'.format(section_name = section, page = page, source = source, start = start, end = end, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = section
    df['image_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHEtiVXw8Wi1tp56Nzd5rH_EoOAJA2RInEWvf5h5CQ-6O_YZp7dw'
    return df

def NYT_api_call_parameter_ALLTIME(param, page, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?q={param}&page={page}&sort=newest&&api-key={api}'.format(param = param, page = page, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = param
    df['image_url'] = 'https://greaterbostonhcs.com/wp-content/uploads/2016/05/Nutrition.jpg'
    return df

def NYT_pull(categories):
    empty = pd.DataFrame()
    for word in categories:
        try:
            df = NYT_api_call_parameter_ALLTIME(word,0,nyt_api_key)
            empty = empty.append(df, sort=True)
            print('Pulled '+word)
            time.sleep(2)
        except:
            print(word + " EXCEPTION!!!!")
    empty = empty.drop(['abstract','section_name'],axis = 1)
    empty = empty.rename(index=str, columns={"_id": "source_id", "document_type": "medium",'pub_date':'date','snippet':'description','word_count':'length'})
    return empty

#NEWSAPI

def rename_columns(df):
    df = df.rename(index=str, columns={'publishedAt':'date','url':'web_url','urlToImage':'image_url'})
    return df

def add_words(df):
    lengths = []
    for string in df.content:
        try:
            lengths.append(round(int(string[string.find('+')+1:string.find(' chars')]) / 4 / 250))
        except:
            lengths.append(4)
    return lengths

def split_source_info(list_of_dicts):
    for item in list_of_dicts:
        item['source_id'] = item['source']['id']
        item['source'] = item['source']['name']

def pull_articles(parameter):
    article_results_rel = newsapi.get_everything(q=parameter,sort_by = 'relevancy',language='en', page_size=10, sources=sources_joined)
    article_results_rel = article_results_rel['articles']
    split_source_info(article_results_rel)
    return article_results_rel

def clean_articles(list_of_dicts, search_param):
    df = pd.DataFrame(list_of_dicts)
    try:
        df['medium'] = 'text'
        df['param'] = search_param
        df['publishedAt'] = df['publishedAt'].apply(lambda x: pd.to_datetime(x).date().strftime('%Y-%m-%d'))
        df['formality'] = 'Intermediate'
        df['length'] = add_words(df)
        df = rename_columns(df)
        print(search_param)
    except:
        pass
    return df


def call_news_api(categories):
    empty_df = pd.DataFrame()
    for category in categories:
        dicts = pull_articles(category)
        df = clean_articles(dicts, category)
        try:
            empty_df = empty_df.append(df, sort=True)
        except:
            pass
        print(len(empty_df.index))
    return empty_df


# Social Media

In [50]:
import tweepy
from apikeys import *
import pandas as pd
import json
import requests
import datetime
from datetime import date
from info import *

auth = tweepy.OAuthHandler(twitter_1, twitter_2)
auth.set_access_token(twitter_3, twitter_4)
api = tweepy.API(auth)
#all twitter handles to scrape
# twitter_handles = ['@ATPScience1', '@waitrose', '@MicrobiomeInst', '@veganrecipescom', '@cldiet', '@Onnit', '@vegsoc', '@VeganKosher', '@TheVeganSociety', '@vegan', '@Keto_Recipes_', '@the52diet', '@IFdiet', '@microbiome', '@metagenomics', '@microbiome_news', '@TheGutStuff', '@MyGutHealth', '@PaleoFX',
# '@PaleoFoundation', '@ThePaleoDiet', '@PaleoComfort', '@cavemanketo', '@KetoFlu', '@TheKetoKitchen_', '@EatKetoWithMe', '@KetoConnect', '@KetoDietZone', '@Ketogenic', '@USDANutrition', '@FoodRev', '@CSPI', '@simplyrecipes', '@FoodNetwork', '@CookingChannel', '@tasty', '@nytfood', '@finecooking', '@mrcookingpanda'
# , '@FODMAPeveryday', '@FODMAPLife', '@FodmappedInfo', '@thefodmapdoctor', '@SimplyGlutenFre', '@gfliving', '@sibotest', '@manjulaskitchen', '@VegTimes', '@CookingLight', '@mealprepwl', '@thehealthygut', '@VitalGutHealth', '@pureguthealth', '@PaleoForBegin', '@PaleoLeap', '@ThePaleoMom', '@paleomagazine', '@PaleoHacks', '@paleogrubs',
# '@naturalgourmet', '@Low_Carb_Keto', '@NutritionTwins', '@mckelhill', '@WomensFitnessAu', '@WomensHealthMag', '@MensHealthMag', '@mjfit', '@thugkitchen', '@Leslie_Klenke', '@insidePN', '@ThisMamaCooks', '@EdibleWildFood', '@TheEarthDieter', '@HarvardHealth', '@EverydayHealth', '@DailyHealthTips']

#clean response from twitter

def clean_tweets(data, categories):
    tweets = []
    for tweet in data:
        try:
            hashtag = tweet.entities['hashtags'][0]['text']
            tags = list(pd.DataFrame(tweet.entities['hashtags']).text)
            intersect = list(set(tags).intersection(categories))
            if len(intersect) > 0:
                hashtag = intersect[0]
            else:
                hashtag = hashtag
        except IndexError:
            hashtag = 'general'
        tweets = [{'title':tweet.id, 'date':tweet.created_at.date().strftime('%Y-%m-%d'),
           'description': tweet.text, 'source':tweet.user.screen_name,'source_id':tweet.user.id_str,
           'formality': 'Informal'
           ,'length': 1,'medium':'text', 'param':hashtag} for tweet in data]
    tweets = pd.DataFrame(tweets)
    return tweets

#CALL API
def twitter_api_call(list_handles, categories):
    empty = pd.DataFrame()
    for handle in list_handles:
        user_tweets = pd.DataFrame(clean_tweets(api.user_timeline(handle), categories))
        empty = empty.append(user_tweets, sort=True)
        print(handle)
    empty.title = empty.title.astype('str')
    empty['web_url'] = 'https://twitter.com/'+empty.source+'/status/'+empty.title
    empty['image_url'] = empty['web_url']
    return empty
import requests

class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text


# Youtube 

In [103]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import pprint
import pafy
import warnings
warnings.filterwarnings("ignore")
from apikeys import *
from info import *


DEVELOPER_KEY = youtube_key
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"


def clean_youtube_time(string):
    if 'H' in string:
        minutes = int(string[string.find('H')+1:string.find('M')])
        hours = int(string[string.find('T')+1:string.find('H')]) * 60
        time = minutes + hours
    else:
        if 'M' in string:
            time = int(string[string.find('T')+1:string.find('M')])
        else:
            time = 1
    return time

def youtube_search(q, max_results=10,order="date", token=None, location=None, location_radius=None):

    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

    search_response = youtube.search().list(
    q=q,
    type="video",
    pageToken=token,
    order = order,
    part="id,snippet", # Part signifies the different types of data you want
    maxResults=max_results,
    location=location,
    locationRadius=location_radius).execute()

    all_dicts = []

    for search_result in search_response.get("items", []):
        if search_result["id"]["kind"] == "youtube#video":

            title = (search_result['snippet']['title'])

            videoId = (search_result['id']['videoId'])

            response = youtube.videos().list(
            part='statistics, snippet, contentDetails',
            id=search_result['id']['videoId']).execute()

            channelId = (response['items'][0]['snippet']['channelId'])
            channelTitle = (response['items'][0]['snippet']['channelTitle'])
            categoryId = (response['items'][0]['snippet']['categoryId'])
            favoriteCount = (response['items'][0]['statistics']['favoriteCount'])
            viewCount = (response['items'][0]['statistics']['viewCount'])
            date = pd.to_datetime((response['items'][0]['snippet']['publishedAt'])).date().strftime('%Y-%m-%d')
            description = response['items'][0]['snippet']['localized']['description']
            url = 'https://www.youtube.com/watch?v='+videoId
            image_url = response['items'][0]['snippet']['thumbnails']['default']['url']
            length = clean_youtube_time(response['items'][0]['contentDetails']['duration'])

        if 'commentCount' in response['items'][0]['statistics'].keys():
            commentCount = (response['items'][0]['statistics']['commentCount'])
        else:
            commentCount = []

        if 'tags' in response['items'][0]['snippet'].keys():
            tags = (response['items'][0]['snippet']['tags'])
        else:
            tags = []

        youtube_dict = {'tags':tags,'source_id': channelId,'source': channelTitle,'categoryId':categoryId,'title':title,'videoId':videoId,'viewCount':viewCount,'commentCount':commentCount,'favoriteCount':favoriteCount,
                        'formality':'Intermediate', 'medium':'video','date':date, 'description': description, 'web_url':url, 'image_url':image_url, 'length':length}
        all_dicts.append(youtube_dict)
    return pd.DataFrame(all_dicts)

def add_category(df, categories):
    # cats = ['keto','ketogenic','paleo','paleolithic','vegan','vegetarian']
    all_params = []
    for index, row in df.iterrows():
        try:
            intersect = list(set(row.tags).intersection(categories))
            if len(intersect) > 0:
                category = intersect[0]
            else:
                category = row.tags[0]
        except:
            category = 'none'
        all_params.append(category)
    df['param'] = all_params
    return df

def youtube_api_call(list_accounts, categories):
    empty_df = pd.DataFrame()
    errors = []
    for account in list_accounts:
        try:
            df = youtube_search(account)
            df = add_category(df, categories)
            empty_df = empty_df.append(df, sort=True)
            print(account)
        except:
            print(account + " EXCEPTION!!!!")
    return empty_df

# FB AND INSTA

In [10]:
# from instagram.client import InstagramAPI
# api = InstagramAPI(client_id="EAACl5okwUhQBAD6vhJiELgsatruedytMV67mvDuN2wgXEyRAXG7umE3T0KEweMhlWQWPgku5pF8KBwwOy9JJuFxx2chbYxPTjMdYA2mTd1DL6Jd5t4XwcetZBwhZC1Pyrl5MW2X90w6J9ZCZCWYlODApHzosU7ReUPZBmFITJj0cpsMZCDZATmEKip23ZA1nyA40Dv8IsLccLAZDZD")
# popular_media = api.media_popular(count=20)
# for media in popular_media:
#     print(media.images['standard_resolution'].url)

In [11]:
import facebook

graph = facebook.GraphAPI(access_token="EAACl5okwUhQBANPTECp6sKqxm3vubZCFmBBEtwdUQgLPAvAFLhpmk6ZAgDTB6Q6g6y6lvz3yjf7iZBW2ELCSrAwDhp28A1xse1QkQrvXowYgivMP0Rz3NRO8cEEy514LC7x3pAmFr1RBZBe7RJVVlsY70OWLHOgceSZCZC0HqzFdw13oEkEZC0Bm4ehPBxTlJFOoyiFE5f8ZCAZDZD", version="2.12")

# AUDIO

In [12]:
import requests
import json
import feedparser
import pandas as pd
import openpyxl

In [13]:
def feedUrlLinks(search_words, media_value='podcast', entity_value='podcast'):
    """
    Fetches FeedUrl(s) of a search request made using the Itunes Api
    
    Args:
        search_words: The URL-encoded text string to be searched for
        media_value: {movie, podcast, music, musicVideo, audiobook,
                    shortFilm, tvShow, software, ebook, all} optional
                    An optional variable, which indicates the media type to be searched for.
        entity_value: Optional
        
    Returns:
        A list of feed urls
        
        example:
        
        ['https://www.npr.org/rss/podcast.php?id=381444908', 
        'http://www.spacemusic.nl/podcast/freshairlounge/freshairlounge.xml', 
        'http://feeds.soundcloud.com/users/soundcloud:users:192634952/sounds.rss', 
        'http://feeds.feedburner.com/FAIOR_vid', 
        'http://feeds.soundcloud.com/users/soundcloud:users:280924508/sounds.rss',
        'http://feeds2.feedburner.com/faior', 'http://rss.acast.com/takingthepulse']

    """
    payload = {'term': search_words, 'media': media_value, 'entity' : entity_value}
    # make a http request with the payload as query parameters
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    # print Full request query
    print("\nYour Passed in query is ->", itunes_request.url)
    
    # Store the json result of the Query
    itunes_result_json = itunes_request.json()
#     return itunes_result_json
    #print("\n",itunes_result_json)
    # Get the number of results count so we know what to loop through
    result_count = itunes_result_json["resultCount"]
    feed_url_list = []
    i = 0
    while(i < result_count):
        # get the Feed Url for each result
        feed_url = itunes_result_json["results"][i]['feedUrl']
        feed_url_list.append(feed_url)
        i = i + 1
    return feed_url_list

In [14]:
feedUrlLinks('wellness mama')


Your Passed in query is -> https://itunes.apple.com/search?term=wellness+mama&media=podcast&entity=podcast


['https://wellnessmama.com/feed/podcast',
 'http://theenginemom.libsyn.com/rss',
 'http://mamainthemaking.libsyn.com/rss',
 'https://unconventionalwellnessradio.castos.com/feed']

In [15]:
# In the future ask the User for the name of the Podcast to be searched for
# construct the Itunes search query to search for the Podcast


feed_url_link_list = feedUrlLinks("wellness mama")

if(len(feed_url_link_list) > 0):
    print("\n", feed_url_link_list)
    i = 1
    for url in feed_url_link_list:
        feed = feedparser.parse(url)
        # (feed.entries) is still the same as feed['entries']
        print('url name', url)
        sheet_list = []
        for item in feed.entries:
            info = dict()
            info['title'] = item['title'] if 'title' in item else ''
            info['Episode description']= item['summary'] if 'summary' in item else ''
            info['Episode length']= item['itunes_duration'] if 'itunes_duration' in item else ''
            info['Release date']= item['published'] if 'published' in item else ''
            sheet_list.append(info)
        df = pd.DataFrame(sheet_list)
        i = i + 1
else:
    print("Search came back as empty, can try again with a different keyword")


Your Passed in query is -> https://itunes.apple.com/search?term=wellness+mama&media=podcast&entity=podcast

 ['https://wellnessmama.com/feed/podcast', 'http://theenginemom.libsyn.com/rss', 'http://mamainthemaking.libsyn.com/rss', 'https://unconventionalwellnessradio.castos.com/feed']
url name https://wellnessmama.com/feed/podcast
url name http://theenginemom.libsyn.com/rss
url name http://mamainthemaking.libsyn.com/rss
url name https://unconventionalwellnessradio.castos.com/feed


In [16]:
feed.entries[0].keys()

dict_keys(['itunes_title', 'title', 'title_detail', 'authors', 'author', 'author_detail', 'subtitle', 'subtitle_detail', 'summary', 'summary_detail', 'content', 'googleplay_description', 'links', 'id', 'guidislink', 'link', 'published', 'published_parsed'])

In [17]:
feed.entries[0]['published_parsed']

time.struct_time(tm_year=2018, tm_mon=11, tm_mday=14, tm_hour=20, tm_min=53, tm_sec=0, tm_wday=2, tm_yday=318, tm_isdst=0)